In [222]:
import re
import codecs
import jieba
import pickle
import pandas as pd
from elasticsearch import Elasticsearch

In [223]:
es = Elasticsearch([{'host':'localhost','port':9200}])

In [224]:
# all_question = pd.read_csv('./data/description.csv')
# all_question = pd.read_excel('./data/Herceptin.xlsx')
all_question = pd.read_excel('./data/Herceptin_funding_filtered.xlsx')
# all_question = pd.read_excel('./data/Herceptin_patient_filtered.xlsx')

In [225]:
all_question.dropna(axis=0, inplace=True, subset=['Description'])

In [226]:
all_question.isna().sum()

Case Number               0
Enquiry Classification    0
Description               0
dtype: int64

In [227]:
jieba.load_userdict('./chinese_word_dict.txt')
chinese_word_dict = pickle.load(open('./chinese_word_dict.pickle', 'rb'))
stopwords = [line.strip() for line in codecs.open('./stopwords-zh.txt', 'r', encoding='utf8').readlines()]

In [228]:
def keyword_extract(text, word_dict):
    text = list(jieba.cut(text, cut_all=False))
    union_text = list(set(text).intersection(set(word_dict)))
    return " ".join(union_text)

In [229]:
all_question['KeyWords'] = all_question.Description.apply(lambda x: keyword_extract(x, chinese_word_dict))

In [230]:
# 去除所有“[]”内的文本
def remove_special_string(text):
    text = re.sub(r'\[[^]]*\]', '', text)
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub(r'（[^）]*\）', '', text)
    text = re.sub(r'\<[^>]*\>', '', text)
    text = re.sub(r'[0-9]*', '', text)
    return text
def remove_stopwords(text, stopwords):
    text_cutted = jieba.cut(text, cut_all=True)
    text_removed = []
    for word in text_cutted:
        if word in stopwords or word.strip()=="":
            continue
        text_removed.append(word)
    return "  ".join(text_removed)

In [231]:
all_question['Processed'] = all_question.Description.apply(lambda x: remove_special_string(x))
all_question['Processed'] = all_question.Processed.apply(lambda x: remove_stopwords(x, stopwords))
all_question.head()

,Case Number,Enquiry Classification,Description,KeyWords,Processed
0,1420656,Stability/Storage/Reconstitution,"赫赛汀 （批号1： N3825B03B3049, 批号2： N3826B02B3050） 配...",使用 赫赛汀,配 之后 结冰 继续 使用
1,1423956,Stability/Storage/Reconstitution,赫赛汀粉末形状如何？患者保存的赫赛汀的稀释液出现结冰情况，是否能否使用（无产品批号信息）,无 使用 患者 赫赛汀,粉末 形状 保存 稀释 稀释液 出现 结冰 冰情 情况 是否 使用
2,1434184,Safety,[Wechat] 请帮忙查一些赫赛汀与血栓栓塞方面的数据和文献（确认无AE）,无 赫赛汀 血栓栓塞 查 文献,帮忙 查 血栓 血栓栓塞 栓塞 方面 数据 和文 文献
3,1436011,Product Quality,拿到的赫赛汀呈现块状是否正常,正常 赫赛汀,拿到 呈现 块状 是否 正常
4,1438071,Product Quality,常规配置和保存的赫赛汀复溶后的溶液保存在2-8度的情况下，现出现浑浊情况，能否使用,溶液 现 赫赛汀 配置 使用 下,常规 配置 保存 复 溶 溶液 保存 存在 度 情况 现出 出现 浑...


In [232]:
doc_list = []
for idx,row in all_question.iterrows():
    doc = {"text": row['Description'], "processed_text": row['Processed']}
    doc_list.append(doc)
print(len(doc_list))
print(doc_list[0])

438
{'text': '赫赛汀 （批号1：\xa0N3825B03B3049, 批号2： N3826B02B3050） 配了之后结冰了， 能否继续使用。', 'processed_text': '配  之后  结冰  继续  使用'}


In [233]:
for idx in range(len(doc_list)):
    es.index(index='index', doc_type='type', id=idx, body=doc_list[idx])

In [221]:
query = ""
results = es.search(index='index', body={'query': {'match':{"text": query}}}, size=10)
hits = results['hits']['hits']
for idx, res in enumerate(hits):
    print(str(idx+1) + ":  " + res['_source']['text'])